In [1]:
import isaac.constants
isaac.constants.TQDM_DISABLE = True

from torch import nn
from torch.nn import Softmax
from isaac.utils import get_cuda_device_if_available
import joblib

from isaac.dataset import read_dataset, prepare_dataset
from isaac.models import MultiBranchModel, ComplexRNNModel
from isaac.constants import BASIC_TRAINING_COLS, MASS_CLASS_COLS, FORCE_CLASS_COLS, RTHETA_COLS, XY_RTHETA_COLS, XY_VXVY_RTHETA_COLS
from isaac.evaluation import predict_with_a_group_of_saved_models, evaluate_saved_model

import torch
import glob
from torch.autograd import Variable
import numpy as np
import pandas as pd
from tqdm import tqdm


In [2]:
SECONDS_PER_WINDOW = 5 # seconds
FPS = 60
STEP_SIZE = 3
# PD_STEP_SIZE = 10
SEQ_END = 2700

In [3]:
device = get_cuda_device_if_available()
print(device)

cpu


In [4]:
HIDDEN_DIM = 25  # hidden layer dimension
N_LAYERS = 4     # number of hidden layers
OUTPUT_DIM = 3   # output dimension
DROPOUT = 0.5

In [5]:
normalise_data = True
model_name = "xy_vxvy"
d = {"rtheta": RTHETA_COLS, "xy_vxvy": BASIC_TRAINING_COLS,
     "xy_rtheta": XY_RTHETA_COLS, "xy_vxvy_rtheta": XY_VXVY_RTHETA_COLS}
training_columns = d[model_name]

dataset_path = "../new_exp_data/exp7_passive.h5"
multiclass = False
DATASET = read_dataset(dataset_path)


def get_question_predictions_for_group_of_models(question_type):    
    
    if not multiclass:
        class_columns = FORCE_CLASS_COLS if question_type == "force" else MASS_CLASS_COLS
        models = sorted(glob.glob("models/GRU_singlebranch/"+model_name+"/best_"+question_type+"_model_seed_*.pt"))
        scaler_path = "scalers/GRU_singlebranch/"+question_type+"_"+model_name+"_scaler.sk"
        model_arch = ComplexRNNModel
        network_dims = (len(training_columns), HIDDEN_DIM, N_LAYERS, OUTPUT_DIM, DROPOUT)

    else:
        class_columns = [list(MASS_CLASS_COLS), list(FORCE_CLASS_COLS)]
        models = sorted(glob.glob("models/"+model_name+"/best_"+question_type+"_model_seed_*.pt"))
        scaler_path = "scalers/passive_"+model_name+"_scaler.sk"
        model_arch = MultiBranchModel
        network_dims = (len(training_columns), HIDDEN_DIM, OUTPUT_DIM, DROPOUT)

    
    predictions = predict_with_a_group_of_saved_models(tqdm(models), network_dims, None, 
                                                       training_columns=training_columns, 
                                                       class_columns=class_columns, step_size=STEP_SIZE, 
                                                       seq_end=SEQ_END, scaler_path=scaler_path,
                                                       arch=model_arch, multiclass=multiclass, trials=DATASET,
                                                       predict_rolling_windows=True, seconds_per_window=SECONDS_PER_WINDOW)
    predictions = torch.stack(predictions)
    if multiclass:
        if question_type == "mass":
            predictions = predictions[:, :, :, 0]
        else:
            predictions = predictions[:, :, :, 1]

    return predictions

# T-test for MASS questions

## Load model's predictions

In [6]:
print("MASS")
question_type = "mass"
group_mass_seq_prediction = get_question_predictions_for_group_of_models(question_type)

print("\nFORCE")
question_type = "force"
group_force_seq_prediction = get_question_predictions_for_group_of_models(question_type)

  0%|          | 0/25 [00:00<?, ?it/s]

MASS


  0%|          | 0/25 [00:00<?, ?it/s]


FORCE


100%|██████████| 25/25 [00:35<00:00,  1.39s/it]


In [7]:
group_mass_seq_prediction.shape

torch.Size([25, 36, 40, 3])

In [8]:
mass_solutions = [trial[list(MASS_CLASS_COLS)].idxmax(axis=1).unique()[0] for trial in DATASET]
force_solutions = [trial[list(FORCE_CLASS_COLS)].idxmax(axis=1).unique()[0] for trial in DATASET]

In [9]:
s = Softmax(dim=-1)
group_force_seq_prediction = s(group_force_seq_prediction)
group_mass_seq_prediction = s(group_mass_seq_prediction)

In [10]:
group_force_seq_prediction.shape

torch.Size([25, 36, 40, 3])

In [11]:
avg_force_seq_prediction = torch.mean(group_force_seq_prediction, dim=0)
avg_mass_seq_prediction = torch.mean(group_mass_seq_prediction, dim=0)

In [12]:
n_trials = avg_force_seq_prediction.shape[0]
n_windows = avg_force_seq_prediction.shape[1]
window_second_start = [i for _ in range(n_trials) for i in range(1, n_windows+1)]
trial_number = [i for i in range(n_trials) for _ in range(1, n_windows+1)]

In [13]:
mass_df = pd.DataFrame(data=avg_mass_seq_prediction.reshape(n_trials*n_windows, 3).numpy(), 
                       columns=["rnn_%s" % cl for cl in MASS_CLASS_COLS])
mass_df["window_second_start"] = window_second_start
mass_df["trial_number"] = trial_number
mass_df["solution"] = [mass_solutions[trial_id] for trial_id in trial_number]

force_df = pd.DataFrame(data=avg_force_seq_prediction.reshape(n_trials*n_windows, 3).numpy(), 
                       columns=["rnn_%s" % cl for cl in FORCE_CLASS_COLS])
force_df["window_second_start"] = window_second_start
force_df["trial_number"] = trial_number
force_df["solution"] = [force_solutions[trial_id] for trial_id in trial_number]

In [14]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(font_scale=3)
sns.set_style("white")

# Plot all Force probabilities throughout the trials

for trial_i in range(force_df["trial_number"].max() + 1):
    trial_df = force_df.query("trial_number == %d" % trial_i)
    
    plt.figure(figsize=(40, 6))
           
    ax = sns.lineplot(y="rnn_attract", x="window_second_start", data=trial_df, linewidth=20, label="attract", markers=True)    
    ax = sns.lineplot(y="rnn_repel", x="window_second_start", data=trial_df, ax=ax, linewidth=20, label="repel")    
    ax = sns.lineplot(y="rnn_none", x="window_second_start", data=trial_df, ax=ax, linewidth=20, label="none")
    sns.despine(ax=ax)
    
    ax.set_ylabel("RNN probability", weight="bold")
    ax.set_yticks([0, 0.5, 1])
    ax.set_yticklabels(ax.get_yticks(), weight="bold")
    ax.set_xlim(0, 45)
    ax.set_xticklabels([])
    ax.set_xlabel("")
    ax.legend(loc=2, prop={'weight':'bold'})
    
    solution = trial_df.solution.unique()[0]
    if solution == "attract":
        ax.lines[1].set_linestyle("--")
        ax.lines[2].set_linestyle("--")
    elif solution == "repel":
        ax.lines[0].set_linestyle("--")
        ax.lines[2].set_linestyle("--")
    else:
        ax.lines[0].set_linestyle("--")
        ax.lines[1].set_linestyle("--")        
        
    plt.savefig("cogsci_images/RNN_rw_all_prob_force_plot_%d.png" % trial_i, bbox_inches='tight')
    plt.close()

for trial_i in range(mass_df["trial_number"].max() + 1):
    trial_df = mass_df.query("trial_number == %d" % trial_i)
    plt.figure(figsize=(40, 6))
           
    ax = sns.lineplot(y="rnn_A", x="window_second_start", data=trial_df, linewidth=20, label="A", markers=True)    
    ax = sns.lineplot(y="rnn_B", x="window_second_start", data=trial_df, ax=ax, linewidth=20, label="B")    
    ax = sns.lineplot(y="rnn_same", x="window_second_start", data=trial_df, ax=ax, linewidth=20, label="same")
    sns.despine(ax=ax)
    
    ax.set_ylabel("RNN probability", weight="bold")
    ax.set_yticks([0, 0.5, 1])
    ax.set_yticklabels(ax.get_yticks(), weight="bold")
    ax.set_xlim(0, 45)
    ax.set_xticklabels([])
    ax.set_xlabel("")
    ax.legend(loc=2, prop={'weight':'bold'})
    
    solution = trial_df.solution.unique()[0]
    if solution == "A":
        ax.lines[1].set_linestyle("--")
        ax.lines[2].set_linestyle("--")
    elif solution == "B":
        ax.lines[0].set_linestyle("--")
        ax.lines[2].set_linestyle("--")
    else:
        ax.lines[0].set_linestyle("--")
        ax.lines[1].set_linestyle("--")        
        
    plt.savefig("cogsci_images/RNN_rw_all_prob_mass_plot_%d.png" % trial_i, bbox_inches='tight')
    plt.close()

# Obtain most confusing intervals
Most confusing = Highest values of probability for a wrong class

In [15]:
def entropy_rnn_responses(responses):
    responses = responses.tolist()
    return -np.sum(responses * np.log2(responses))

def get_probabilities_df(df, task):
    rows = []
    
    class_columns = FORCE_CLASS_COLS if task == "force" else MASS_CLASS_COLS
    rnn_columns = ["rnn_" + cl for cl in class_columns]

    
    for i in range(df.shape[0]):
        row = df.iloc[i]
        window_second_start = row.window_second_start
        trial_number = row.trial_number
        solution = row.solution
        
        rnn_solution_probability = row["rnn_"+solution]
        rnn_preferred_option = class_columns[np.argmax(row[rnn_columns].values)]
        rnn_preferred_option_probability = max(row[rnn_columns])
        rnn_entropy = entropy_rnn_responses(row[rnn_columns].values)
        
        wrong_options = ["rnn_" + cl for cl in class_columns if cl != solution]
        max_probability_for_a_wrong_option = max(row[wrong_options])
        
        
        rows.append([window_second_start, trial_number, solution,
                    rnn_solution_probability, rnn_preferred_option, rnn_preferred_option_probability,
                    rnn_entropy, max_probability_for_a_wrong_option])
        

    df = pd.DataFrame(data=rows, columns=["window_second_start", "trial_number", "solution", "rnn_solution_probability",
                                         "rnn_preferred_option", "rnn_preferred_option_probability", "entropy",
                                         "max_probability_for_a_wrong_option"])
    return df


In [16]:
confused_mass_dfs = get_probabilities_df(mass_df, "mass")
confused_force_dfs = get_probabilities_df(force_df, "force")

In [17]:
confused_mass_dfs = confused_mass_dfs.sort_values(by="max_probability_for_a_wrong_option", ascending=False)
confused_force_dfs = confused_force_dfs.sort_values(by="max_probability_for_a_wrong_option", ascending=False)

## Load human results

In [18]:
from isaac.visualization import make_frame_curried
import moviepy.editor as mpy
from scipy import misc


def make_clip(trial_data, window_second_start, solution, rnn_thinks_this_is, rnn_confidence):
            
    trial_data = trial_data.iloc[window_second_start*FPS:(window_second_start + SECONDS_PER_WINDOW)*FPS]
    duration = len(trial_data)
    n_bodies = sum(["o"+str(i)+".x" in list(trial_data.columns) for i in range(1, 5)])
    
    while (len(trial_data) + 1) % 60 != 0:
        trial_data = trial_data.append(trial_data.iloc[-1], ignore_index=True)
        
    make_frame = make_frame_curried(trial_data, n_bodies, None, None)
    clip = mpy.VideoClip(make_frame, duration=duration / 60)
    return clip, trial_data

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [19]:
%load_ext autoreload
%autoreload 2

# Create videos for comparing evidence seen by RNN and by IO

In [20]:
import os
import json

CONFUSING_DATA_PATH = "cogsci_images/confusing_videos/confusing_%s" % model_name
os.makedirs(CONFUSING_DATA_PATH, exist_ok=True)
FILENAME = "confusing_%s_interval_in_trial_%d_sec_%d_to_%d_rnn_thinks_%s__prob_%.4f_while_solution_is_%s_prob_%.4f"
VIDEO_PATH = os.path.join(CONFUSING_DATA_PATH, FILENAME + ".mp4")
JSON_PATH = os.path.join(CONFUSING_DATA_PATH, "confusing_"+model_name+"_%s_physics_data.json")
JSON_DATA = []

def write_confused_intervals(confused_df, question_type, solution):

    replays = read_dataset("../new_exp_data/exp7_passive.h5")
    print(len(replays))
    written_replays = {}
    number_of_written_replays = 0
    
    
    confused_df = confused_df.query("solution == '%s'" % solution).copy()

    for row_i in range(confused_df.shape[0]):

        window_second_start, trial_number, solution, rnn_preferred_option, rnn_preferred_option_probability, rnn_solution_probability = (
            confused_df.iloc[row_i][["window_second_start", "trial_number", "solution", "rnn_preferred_option",
                                     "rnn_preferred_option_probability", "rnn_solution_probability"]])

        window_second_end = window_second_start + SECONDS_PER_WINDOW
        
        print("RNN thinks the interval (%d, %d) in trial %d is %s with %.4f confidence. In reality, it is %s (%.4f)." % (
            window_second_start, window_second_end, trial_number,
            rnn_preferred_option, rnn_preferred_option_probability, solution, rnn_solution_probability))
        
        
        # DON'T SAVE AN INTERVAL IF IT REFRESHES
        window_frame_start = window_second_start * 60
        window_frame_end = window_second_end * 60
        frames_in_which_the_replay_refreshes = replays[trial_number].refreshes[0] if\
                                               "refreshes" in replays[trial_number].columns\
                                                else []
        frames_in_which_the_replay_refreshes = [frames_in_which_the_replay_refreshes] if\
                                                    type(frames_in_which_the_replay_refreshes) == np.int64\
                                                else frames_in_which_the_replay_refreshes
        
        interval_refreshes = False
        for frame in frames_in_which_the_replay_refreshes:
            if window_frame_end > frame > window_frame_start:
                interval_refreshes = True
                break
        if interval_refreshes:
            print("Skipping trial %d and start %d because of interval refreshes" % (trial_number, window_second_start))
            continue            
        
        # DON'T SAVE AN INTERVAL IF THERE'S AN OVERLAPPING INTERVAL ALREADY SAVED
        if trial_number in written_replays:
            overlapping_replay = False
            for already_written_window_start in written_replays[trial_number]:
                if abs(already_written_window_start - window_second_start) <= 5:
                    overlapping_replay = True
                    break
            if overlapping_replay:
                print("Skipping trial %d and start %d because of interval overlap" % (trial_number, window_second_start))
                continue

        # SAVE THE INTERVAL
        written_replays[trial_number] = written_replays.get(trial_number, []) + [window_second_start]
        clip, trial_data = make_clip(replays[trial_number], window_second_start, solution, rnn_preferred_option, rnn_preferred_option_probability)
        clip.ipython_display(fps=60)
        clip.write_videofile(VIDEO_PATH % (
            question_type, trial_number, window_second_start, window_second_end,
            rnn_preferred_option, rnn_preferred_option_probability, solution, rnn_solution_probability), fps=60)

        trial_data = trial_data.to_dict(orient='list')
        # Simplify attributes whose values are unique throughout the list
        for key in ["trial_type", "condition_world_variant", "tM", "tR", "world_id"]:
            trial_data[key] = trial_data[key][0]
        JSON_DATA.append(trial_data)

        number_of_written_replays += 1
        if number_of_written_replays == 5:
            break
    
    print(number_of_written_replays)

In [21]:
write_confused_intervals(confused_force_dfs, "force", "attract")
write_confused_intervals(confused_force_dfs, "force", "repel")
write_confused_intervals(confused_force_dfs, "force", "none")

with open(JSON_PATH % "force", "w+") as f: 
    json.dump(JSON_DATA, f)

36
RNN thinks the interval (25, 30) in trial 1 is none with 0.7070 confidence. In reality, it is attract (0.1140).
Skipping trial 1 and start 25 because of interval refreshes
RNN thinks the interval (10, 15) in trial 28 is repel with 0.6895 confidence. In reality, it is attract (0.0215).


100%|█████████▉| 300/301 [00:02<00:00, 134.16it/s]

[MoviePy] >>>> Building video cogsci_images/confusing_videos/confusing_xy_vxvy/confusing_force_interval_in_trial_28_sec_10_to_15_rnn_thinks_repel__prob_0.6895_while_solution_is_attract_prob_0.0215.mp4
[MoviePy] Writing video cogsci_images/confusing_videos/confusing_xy_vxvy/confusing_force_interval_in_trial_28_sec_10_to_15_rnn_thinks_repel__prob_0.6895_while_solution_is_attract_prob_0.0215.mp4



100%|█████████▉| 300/301 [00:02<00:00, 128.37it/s]

[MoviePy] Done.
[MoviePy] >>>> Video ready: cogsci_images/confusing_videos/confusing_xy_vxvy/confusing_force_interval_in_trial_28_sec_10_to_15_rnn_thinks_repel__prob_0.6895_while_solution_is_attract_prob_0.0215.mp4 



...

100%|█████████▉| 300/301 [00:02<00:00, 132.57it/s]


[MoviePy] >>>> Building video cogsci_images/confusing_videos/confusing_xy_vxvy/confusing_force_interval_in_trial_21_sec_17_to_22_rnn_thinks_repel__prob_0.6117_while_solution_is_none_prob_0.3396.mp4
[MoviePy] Writing video cogsci_images/confusing_videos/confusing_xy_vxvy/confusing_force_interval_in_trial_21_sec_17_to_22_rnn_thinks_repel__prob_0.6117_while_solution_is_none_prob_0.3396.mp4


100%|█████████▉| 300/301 [00:02<00:00, 129.10it/s]

[MoviePy] Done.
[MoviePy] >>>> Video ready: cogsci_images/confusing_videos/confusing_xy_vxvy/confusing_force_interval_in_trial_21_sec_17_to_22_rnn_thinks_repel__prob_0.6117_while_solution_is_none_prob_0.3396.mp4 



5


In [22]:
JSON_DATA = []

write_confused_intervals(confused_mass_dfs, "mass", "A")
write_confused_intervals(confused_mass_dfs, "mass", "B")
write_confused_intervals(confused_mass_dfs, "mass", "same")

with open(JSON_PATH % "mass", "w+") as f: 
    json.dump(JSON_DATA, f)

36
RNN thinks the interval (39, 44) in trial 25 is B with 0.6947 confidence. In reality, it is A (0.0476).


100%|█████████▉| 300/301 [00:02<00:00, 132.69it/s]


[MoviePy] >>>> Building video cogsci_images/confusing_videos/confusing_xy_vxvy/confusing_mass_interval_in_trial_25_sec_39_to_44_rnn_thinks_B__prob_0.6947_while_solution_is_A_prob_0.0476.mp4
[MoviePy] Writing video cogsci_images/confusing_videos/confusing_xy_vxvy/confusing_mass_interval_in_trial_25_sec_39_to_44_rnn_thinks_B__prob_0.6947_while_solution_is_A_prob_0.0476.mp4


100%|█████████▉| 300/301 [00:02<00:00, 125.44it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: cogsci_images/confusing_videos/confusing_xy_vxvy/confusing_mass_interval_in_trial_25_sec_39_to_44_rnn_thinks_B__prob_0.6947_while_solution_is_A_prob_0.0476.mp4 

RNN thinks the interval (1, 6) in trial 19 is B with 0.6699 confidence. In reality, it is A (0.0716).


...

100%|█████████▉| 300/301 [00:02<00:00, 121.60it/s]


[MoviePy] >>>> Building video cogsci_images/confusing_videos/confusing_xy_vxvy/confusing_mass_interval_in_trial_21_sec_31_to_36_rnn_thinks_A__prob_0.7163_while_solution_is_same_prob_0.2397.mp4
[MoviePy] Writing video cogsci_images/confusing_videos/confusing_xy_vxvy/confusing_mass_interval_in_trial_21_sec_31_to_36_rnn_thinks_A__prob_0.7163_while_solution_is_same_prob_0.2397.mp4


100%|█████████▉| 300/301 [00:02<00:00, 121.82it/s]

[MoviePy] Done.
[MoviePy] >>>> Video ready: cogsci_images/confusing_videos/confusing_xy_vxvy/confusing_mass_interval_in_trial_21_sec_31_to_36_rnn_thinks_A__prob_0.7163_while_solution_is_same_prob_0.2397.mp4 



5
